In [6]:
#external files
from preprocessing import FileIO
from weaviate_interface import WeaviateClient, WeaviateIndexer
from class_templates import impact_theory_class_properties

#standards
import os
import time
import json
from typing import List
from tqdm.notebook import tqdm

from rich import print  # nice library that provides improved printing output (overrides default print function)

#load from local .env file
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [2]:
#read env vars from local .env file
api_key = os.environ['WEAVIATE_API_KEY']
url = os.environ['WEAVIATE_ENDPOINT']

#instantiate client
client = WeaviateClient(api_key, url)

#check if WCS instance is live and ready
client.is_live(), client.is_ready()

(True, True)

In [3]:
data_path = '../impact-theory-newft-256.parquet'

data = FileIO().load_parquet(data_path)

Shape of data: (26448, 12)
Memory Usage: 2.42+ MB


In [7]:
class_name = 'NEW_FT_INDEX'

In [8]:
class_config = {'classes': [

                      {"class": class_name,        
                       
                       "description": "Episodes of Impact Theory up to Nov 2023", 
                       
                       "vectorIndexType": "hnsw", 
                       
                       # Vector index specific settings
                       "vectorIndexConfig": {                   
                          
                        "ef": 400,
                        "efConstruction": 128, 
                        "maxConnections": 32,  
                        "dynamicEfMin": 100,   
                        "dynamicEfMax": 400,    
                                            },
                       
                       "vectorizer": "none",            
                       
                       # pre-defined property mappings
                       "properties": impact_theory_class_properties }         
                      ]
               }

In [21]:
hnsw_configs = ['maxConnections', 'efConstruction', 'ef']
{k:v for k,v in client.show_class_config(class_name)['vectorIndexConfig'].items() if k in hnsw_configs}

{'maxConnections': 32, 'efConstruction': 128, 'ef': 400}

Bad pipe message: %s [b"(\xe0\xa5\x9dT\xb2\x9b\xe8\xe8\xd2\x9f(\x89\xccA\x8co\xb8\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d"]
Bad pipe message: %s [b'O\xe8J\xe0S{u\x1a\xbe\x18%\xff\x16\xb8#(\x8a\xb7\x00\x00\xa2\xc0\x14\xc0\n\x009\x008\x007\x006\x00', b'\x87\x00\x86\x00\x85\xc0\x19\x00:\x00\x89\xc0\x0f\xc0\x05\x005\x00\x84\xc0\x13\xc0\t\x003\x002\x001\x000\x00\x9a\x00\x99\x00\x98\x00\x97\x00E\x00D\x00C\x00B\xc0\x18\x004\x00\x9b\x00F\xc0\x0e\xc0\x04\x00/\x00\x96\x00A\x00\x07\xc0\x11\xc0\x07\xc0\x16\x00\x18\xc0\x0c\xc0\x02\x00\x05\x00\x04\xc0\x12\xc0\x08\x00\x16\x00\x13\x00\x10\x00\r\xc0\x17\x00\x1b\xc0\r\xc0\x03\x00\n\x00\x15\x00\x12\x00\x0f\x00\x0c\x00\x1a\x00\t\x00\x14\x00\x11\x00\x19\x00\x08\x00\x06\x00\x17\x00\x03\xc0\x10\xc0\x0

In [9]:
client.schema.create(class_config)

In [10]:
indexer = WeaviateIndexer(client, batch_size=200, num_workers=4)

In [11]:
indexer.batch_index_data(data, class_name)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 26448/26448 [01:30<00:00, 291.10it/s]


Batch job completed in 1.57 minutes.
{'class': 'NEW_FT_INDEX', 'name': 'Vfnhci7XhOpE', 'objectCount': 26448, 'vectorIndexingStatus': 'READY', 'vectorQueueLength': 0}
